In [2]:
import pandas as pd
import openai

# Set your OpenAI API key
openai.api_key = "your-openai-api"

# Function to load data from CSV files
def load_data(demographics_file, transactions_file):
    demographics_df = pd.read_csv(demographics_file)
    transactions_df = pd.read_csv(transactions_file)
    return demographics_df, transactions_df

# Function to analyze transactions
def analyze_transactions(transactions):
    transactions['Date'] = pd.to_datetime(transactions['Date'])
    transactions['Month'] = transactions['Date'].dt.month
    transactions['Year'] = transactions['Date'].dt.year
    
    # Calculate monthly expenses
    monthly_expenses = transactions.groupby(['Year', 'Month'])['Withdraw Amount'].sum().reset_index()
    monthly_expenses.columns = ['Year', 'Month', 'Total Withdraw']
    
    # Calculate monthly deposits
    monthly_deposits = transactions.groupby(['Year', 'Month'])['Deposit Amount'].sum().reset_index()
    monthly_deposits.columns = ['Year', 'Month', 'Total Deposit']
    
    # Merge expenses and deposits
    monthly_summary = pd.merge(monthly_expenses, monthly_deposits, on=['Year', 'Month'], how='outer')
    monthly_summary.fillna(0, inplace=True)
    
    return monthly_summary

# Function to analyze demographics
def analyze_demographics(demographics, client_id):
    client_data = demographics[demographics['Client ID'] == client_id]
    
    if client_data.empty:
        return None
    
    user_info = {
        'Age': client_data['Age'].values[0],
        'Marital Status': client_data['Marital Status'].values[0],
        'Number of Children': client_data['Number of Children'].values[0],
        'Income Level': client_data['Income Level'].values[0],
        'Employment Status': client_data['Employment Status'].values[0],
        'Education Level': client_data['Education Level'].values[0],
        'Debt Level': client_data['Debt Level'].values[0],
        'Housing Status': client_data['Housing Status'].values[0],
        'Health Expenses': client_data['Health Expenses'].values[0],
        'Lifestyle Choices': client_data['Lifestyle Choices'].values[0]
    }
    return user_info

# Class to generate recommendations using GPT-3.5 Turbo
from openai import OpenAI

client=OpenAI(api_key="your-openai-api",)
class FinancialChatbot:
    def __init__(self, model, openai_api_key):
        self.model = model
        self.openai_api_key = openai_api_key
        openai.api_key = openai_api_key

    def generate_recommendations(self, transactions_summary, user_info):
        prompt = f"""
        Stop asking to consult any financial advisor. You are a professional financial advisor. Your name is Finn. Based on the user's transactions and demographics, go very very deep inside the data, provide personalized financial advice; always add numbers like how much to save and where; how much to, how much not to spend in which category, how much to invest and where. Always, always quantify your suggestions.
        
        User Demographics:
        - Age: {user_info['Age']}
        - Marital Status: {user_info['Marital Status']}
        - Number of Children: {user_info['Number of Children']}
        - Income Level: {user_info['Income Level']}
        - Employment Status: {user_info['Employment Status']}
        - Education Level: {user_info['Education Level']}
        - Debt Level: {user_info['Debt Level']}
        - Housing Status: {user_info['Housing Status']}
        - Health Expenses: {user_info['Health Expenses']}
        - Lifestyle Choices: {user_info['Lifestyle Choices']}
        
        Monthly Transaction Summary:
        {transactions_summary.to_string(index=False)}
        
        Recommendations:
        """
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
        )
        
        return response.choices[0].message.content
    
    def chat(self, user_id, demographics_df, transactions_df):
        client_demographics = demographics_df[demographics_df['Client ID'] == user_id]
        client_transactions = transactions_df[transactions_df['Client ID'] == user_id]
        
        if client_demographics.empty or client_transactions.empty:
            return "Client ID not found in the dataset."
        
        # Analyze the data
        transactions_summary = analyze_transactions(client_transactions)
        user_info = analyze_demographics(client_demographics, user_id)
        
        # Generate recommendations
        recommendations = self.generate_recommendations(transactions_summary, user_info)
        print("Financial Recommendations:")
        print(recommendations)
        
        while True:
            query = input("\nYou: ")
            if query.lower() in ["exit", "quit"]:
                print("Goodbye!")
                break
            
            # Generate GPT-3.5 Turbo response
            prompt = f"User said: {query}\nYou are a financial advisor. Your name is Finn. Never ask to consult any other financial advisor. Provide a response based on the given data, user's responses, and financial recommendations- always quantify and add numbers: {recommendations}"
            response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
        )
        
            
            bot_response = response.choices[0].message.content
            print(f"Finn: {bot_response}")

# Main function for testing
if __name__ == "__main__":
    # File paths to the CSV files
    demographics_file = 'bank_clients_demographics.csv'
    transactions_file = 'client_transactions.csv'
    
    # Load the data
    demographics_df, transactions_df = load_data(demographics_file, transactions_file)
    
    # Take client ID from user input
    client_id = int(input("Enter Client ID: "))
    
    # Create an instance of the chatbot
    chatbot = FinancialChatbot("gpt-3.5-turbo", openai.api_key)
    
    # Start the chat
    chatbot.chat(client_id, demographics_df, transactions_df)


Enter Client ID:  2


C:\Users\anany\AppData\Local\Temp\ipykernel_3340\1225697401.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions['Date'] = pd.to_datetime(transactions['Date'])
C:\Users\anany\AppData\Local\Temp\ipykernel_3340\1225697401.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions['Month'] = transactions['Date'].dt.month
C:\Users\anany\AppData\Local\Temp\ipykernel_3340\1225697401.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

Financial Recommendations:
1. Increase Emergency Savings: Based on your transaction history, it looks like your monthly expenses vary significantly. It is important to have at least 3-6 months' worth of expenses saved up in an emergency fund. Aim to save an additional $5,000 per month towards building up this fund.

2. Reduce Discretionary Spending: Your lifestyle choices seem to be on the higher side, which may be contributing to your high debt level. Consider cutting back on non-essential expenses such as dining out, shopping, and entertainment. Try to save at least $1,000 per month by reducing discretionary spending.

3. Pay Down High-Interest Debt: Since your debt level is high, focus on paying off high-interest debts such as credit card balances or personal loans. Allocate an extra $2,000 per month towards paying down these debts to reduce the overall interest costs.

4. Increase Retirement Savings: As a middle-income earner, it's important to prioritize saving for retirement. Aim


You:  should i buy a car worth 50k dollars right now?


Finn: Based on the personalized financial recommendations provided, it would be advisable to reconsider purchasing a car worth $50,000 at this time. Instead, it is important to focus on increasing emergency savings, reducing discretionary spending, paying down high-interest debt, increasing retirement savings, investing in education savings for children, and reviewing insurance policies. By following these steps, you can improve your overall financial health and work towards achieving your long-term financial goals. If you have any further questions or need additional guidance, please feel free to ask.



You:  exit


Goodbye!
